### Function Decorator Basic


In [1]:
# Assigning functions to variables
def plus_one(number):
    return number + 1

add_one = plus_one
add_one(5)

6

In [2]:
# Defining functions inside other functions
def plus_one(number):
    def add_one(number):
        return number + 1


    result = add_one(number)
    return result
plus_one(4)

5

In [3]:
# Passing functions as arguments
def plus_one(number):
    return number + 1

def function_call(function):
    number_to_add = 5
    return function(number_to_add)

function_call(plus_one)

6

In [4]:
# Functions returning other functions
def hello_function():
    def say_hi():
        return "Hi"
    return say_hi
hello = hello_function()
hello()

'Hi'

In [5]:
# Nested function has access to variable scope of enclosing function
def print_message(message):
    # Enclosong Function
    def message_sender():
        # Nested Function
        print(message)

    message_sender()

print_message("Some random message")

Some random message


In [6]:
# Defining a decorator, it takes function as an argument and do the manupulations in it.
# The manupulations are done in wrapper function which is then returned 
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase

    return wrapper

In [7]:
# @uppercase_decorator is equivalent to uppercase_decorator(function)
def say_hi():
    return 'hello there'

decorate = uppercase_decorator(say_hi)
decorate()

'HELLO THERE'

In [8]:
@uppercase_decorator
def say_hi():
    return 'hello there'

say_hi()

'HELLO THERE'

In [9]:
# Multiple decorators
def split_string(function):
    def wrapper():
        func = function()
        splitted_string = func.split()
        return splitted_string

    return wrapper

@split_string
@uppercase_decorator
def say_hi():
    return 'hello there'
say_hi()

['HELLO', 'THERE']

In [10]:
# Accepting arguments in functions being deorated
def decorator_with_arguments(function):
    def wrapper_accepting_arguments(arg1, arg2):
        print("My arguments are: {0}, {1}".format(arg1,arg2))
        function(arg1, arg2)
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one, city_two):
    print("Cities I love are {0} and {1}".format(city_one, city_two))

cities("Patna", "Bangalore")

My arguments are: Patna, Bangalore
Cities I love are Patna and Bangalore


In [11]:
# General purpose decorator, which takes in functions with all the arguments
def a_decorator_passing_arbitrary_arguments(function):
    def a_wrapper_accepting_arbitrary_arguments(*args,**kwargs):
        print('The positional arguments are', args)
        print('The keyword arguments are', kwargs)
        function(*args, **kwargs)
    return a_wrapper_accepting_arbitrary_arguments

In [12]:
@a_decorator_passing_arbitrary_arguments
def func(middle_name, first_name, last_name):
    print(f"Full name is {first_name} {middle_name} {last_name}")

func("YOYO", first_name="Utkarsh", last_name="Prakash")

The positional arguments are ('YOYO',)
The keyword arguments are {'first_name': 'Utkarsh', 'last_name': 'Prakash'}
Full name is Utkarsh YOYO Prakash


### Class Decorators

In [13]:
# Position Class definition
def typename(obj):
    return type(obj).__name__

class Position:

    def __init__(self, latitude, longitude):
        if not (-90 <= latitude <= +90):
            raise ValueError(f"Latitude {latitude} out of range")

        if not (-180 <= longitude <= +180):
            raise ValueError(f"Longitude {longitude} out of range")

        self._latitude = latitude
        self._longitude = longitude

    @property
    def latitude(self):
        return self._latitude

    @property
    def longitude(self):
        return self._longitude

    @property
    def latitude_hemisphere(self):
        return "N" if self.latitude >= 0 else "S"

    @property
    def longitude_hemisphere(self):
        return "E" if self.longitude >= 0 else "W"

    def __repr__(self):
        return f"{typename(self)}(latitude={self.latitude}, longitude={self.longitude})"

    def __str__(self):
        return format(self)

    def __format__(self, format_spec):
        component_format_spec = ".2f"
        prefix, dot, suffix = format_spec.partition(".")
        if dot:
            num_decimal_places = int(suffix)
            component_format_spec = f".{num_decimal_places}f"
        latitude = format(abs(self.latitude), component_format_spec)
        longitude = format(abs(self.longitude), component_format_spec)
        return (
            f"{latitude}° {self.latitude_hemisphere}, "
            f"{longitude}° {self.longitude_hemisphere}"
        )

class EarthPosition(Position):
    pass

class MarsPosition(Position):
    pass

In [14]:
# This class takes in name of the city and its location instance as argument
# Its __str__ is defined, but __repr__ is not defined
class Location:

    def __init__(self, name, position):
        self._name = name
        self._position = position

    @property
    def name(self):
        return self._name

    @property
    def position(self):
        return self._position

    def __str__(self):
        return self.name

hong_kong = Location("Hong Kong", EarthPosition(22.29, 114.16))
stockholm = Location("Stockholm", EarthPosition(59.33, 18.06))
cape_town = Location("Cape Town", EarthPosition(-33.93, 18.42))
rotterdam = Location("Rotterdam", EarthPosition(51.96, 4.47))
maracaibo = Location("Maracaibo", EarthPosition(10.65, -71.65))

In [15]:
hong_kong.position

EarthPosition(latitude=22.29, longitude=114.16)

In [16]:
# Lets define a decorator which will synthesize a __repr__ method for the class
# For revision, a __repr__ method should regenerate the class call format

# A class decorator should intake a class and output a class
def auto_repr(cls):
    print(f"Decorating {cls.__name__} with auto_repr")
    members = vars(cls)
    for name, member in members.items():
        print(name, member)
    return cls

@auto_repr
class Location:
    def __init__(self, name, position):
        self._name = name
        self._position = position

    @property
    def name(self):
        return self._name

    @property
    def position(self):
        return self._position

    def __str__(self):
        return self.name

Decorating Location with auto_repr
__module__ __main__
__init__ <function Location.__init__ at 0x000002ACE2163708>
name <property object at 0x000002ACE215F188>
position <property object at 0x000002ACE2152688>
__str__ <function Location.__str__ at 0x000002ACE21638B8>
__dict__ <attribute '__dict__' of 'Location' objects>
__weakref__ <attribute '__weakref__' of 'Location' objects>
__doc__ None


The output tells us that decorators are called when class is defined.

In [17]:
# Before we can synthesize the __repr__ we need to perform few necessary checks in the decorator.

import inspect

def auto_repr(cls):
    print(f"Decorating {cls.__name__} with auto_repr")
    members = vars(cls)
    for name, member in members.items():
        print(name, member)

    #1 __repr__ should not be already defined in the class
    if "__repr__" in members:
        raise TypeError(f"{cls.__name__} already defines __repr__")

    #2 The decorated class should have its own initializer.
    # We are trying to replicate class call which we can only do if know what __init__ accepts.
    if "__init__" not in members:
        raise TypeError(f"{cls.__name__} does not override __init__")

    #3 All the parameters passed to __init__ must have a corressponding property defined
    sig = inspect.signature(cls.__init__)
    # Excluding self, which is the first parameter to init
    parameter_names = list(sig.parameters)[1:]
    print("__init__ parameter names: ", parameter_names)

    # Checking if there exist a property for each parameter
    if not all(
        isinstance(members.get(name, None), property)
        for name in parameter_names
    ):
        raise TypeError(
            f"Cannot apply auto_repr to {cls.__name__} because not all "
            "__init__ parameters have matching properties"
        )

    return cls


@auto_repr
class Location:
    def __init__(self, name, position):
        self._name = name
        self._position = position

    @property
    def name(self):
        return self._name

    @property
    def position(self):
        return self._position

    def __str__(self):
        return self.name

Decorating Location with auto_repr
__module__ __main__
__init__ <function Location.__init__ at 0x000002ACE21634C8>
name <property object at 0x000002ACE217D7C8>
position <property object at 0x000002ACE217E138>
__str__ <function Location.__str__ at 0x000002ACE2172168>
__dict__ <attribute '__dict__' of 'Location' objects>
__weakref__ <attribute '__weakref__' of 'Location' objects>
__doc__ None
__init__ parameter names:  ['name', 'position']


In [18]:
# Synthesizing the __repr__ method
def auto_repr(cls):
    members = vars(cls)

    if "__repr__" in members:
        raise TypeError(f"{cls.__name__} already defines __repr__")

    if "__init__" not in members:
        raise TypeError(f"{cls.__name__} does not override __init__")

    sig = inspect.signature(cls.__init__)
    parameter_names = list(sig.parameters)[1:]

    if not all(
        isinstance(members.get(name, None), property)
        for name in parameter_names
    ):
        raise TypeError(
            f"Cannot apply auto_repr to {cls.__name__} because not all "
            "__init__ parameters have matching properties"
        )

    def synthesized_repr(self):
        # we are using self and not cls as to get an instance of calling class and not base class
        return "{typename}({args})".format(
            typename=typename(self),
            args=", ".join(
                "{name}={value!r}".format(
                    name=name,
                    value=getattr(self, name)
                ) for name in parameter_names
            )
        )

    # Setting the __repr__ attribute for cls
    setattr(cls, "__repr__", synthesized_repr)

    return cls


@auto_repr
class Location:
    def __init__(self, name, position):
        self._name = name
        self._position = position

    @property
    def name(self):
        return self._name

    @property
    def position(self):
        return self._position

    def __str__(self):
        return self.name

In [19]:
hong_kong = Location("Hong Kong", EarthPosition(22.29, 114.16))
stockholm = Location("Stockholm", EarthPosition(59.33, 18.06))
cape_town = Location("Cape Town", EarthPosition(-33.93, 18.42))
rotterdam = Location("Rotterdam", EarthPosition(51.96, 4.47))
maracaibo = Location("Maracaibo", EarthPosition(10.65, -71.65))
# Lets check if we can get the __repr__ result without defining the method in class
hong_kong

Location(name='Hong Kong', position=EarthPosition(latitude=22.29, longitude=114.16))